# Importing Libraries

In [0]:
!pip install google-cloud-vision


In [34]:

import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt
%matplotlib inline

import os
import os.path
import numpy as np
import pandas as pd      
import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split
import pickle
from sklearn.ensemble import RandomForestClassifier
import io
import cv2
from google.auth import compute_engine
from google.cloud import vision
from googleapiclient.discovery import build
from google.cloud import translate
import six
import urllib.request
from PIL import Image
from google.colab import drive
from googleapiclient import errors

from __future__ import print_function
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
from httplib2 import Http
from oauth2client import file, client, tools
from email.mime.text import MIMEText
from base64 import urlsafe_b64encode



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Initiliasing Variables

In [23]:

''' FILES ARE IN THE SHARED FOLDER'''

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.compose']


drive.mount('/content/gdrive')     #Mount google drive
#proj_folder = 'gdrive/My Drive/RCCB/' #My folder in which all my files are stored
proj_folder = 'gdrive/My Drive/Colab Notebooks/'  

nltk.data.path.append(proj_folder)   #Tell nlt to look into proj_folder for files
eng_stopwords = stopwords.words('english')  #define stopwords
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = proj_folder+'RCCB-cred.json'  #Load credential files from proj_folder

gmail_cred_files = proj_folder+'gmail-credentials.json'
token_fl = proj_folder+'token.pickle'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Vision - Detect Text

  detect_text(path)

In [0]:



def detect_text(path):
    """Detects text in the image."""
    
    client = vision.ImageAnnotatorClient()  #Intitialize vision client
    try:

      with io.open(path, 'rb') as image_file: #open image
        content = image_file.read()
    except FileNotFoundError:     #If image url then get the image
      try: #Check of the variable is an url 
        cont=Image.open(urllib.request.urlopen(path))
        imgByteArr = io.BytesIO()
        cont.save(imgByteArr, format='PNG')
        content = imgByteArr.getvalue()
      except ValueError:  #Otherwise if not an url then print  File not found
        print("File not found. Please mention a valid file")
        return None
      
    image = vision.types.Image(content=content)  #Convert image to vision file

    response = client.text_detection(image=image)  #Get text in the image
    texts = response.text_annotations
    text_gen = texts[0].description #Getting the text
    
    
    '''
    print('Texts:')                                                       FOR GETTING VERTICES
    for text in texts:
        print('\n"{}"'.format(text.description))
        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))'''

    return text_gen

# Translating text
  text_translate(text)



In [0]:
def text_translate(text):
  translate_client = translate.Client()  #Initialize the client

  if isinstance(text, six.binary_type):
      text = text.decode('utf-8')  

# Text can also be a sequence of strings, in which case this method
# will return a sequence of results for each text.
  result = translate_client.translate(
      text, target_language='eng')
  return result['translatedText']

# GetText(filepath) - Combines Translate and get Text

In [0]:
def GetText(filepath):           #function for combining translate and text detection
  det_text = detect_text(filepath) 
  print(det_text)
  if det_text is not None:
    eng_text = text_translate(det_text)
    return eng_text
  else:
    print("File not valid")
    return ""

# Unknown - ADD COMMENTS

In [12]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())


     |████████████████████████████████| 993kB 5.0MB/s 


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

NameError: ignored

# Test Text detection and Translation

In [10]:
#@title Calling Text Detection & Translation Function 'GetText()'{ vertical-output: true}
#@markdown Enter the image url or local file path it should be in proj_folder 
FileURL = "NoWater.jpg" #@param {type:"string"}
FileURL = 'gdrive/My Drive/RCCB/'+FileURL
a = GetText(FileURL)
#@markdown Result:
print(a)

मेरे पस
नी नही है।
पा

I do not have pain Pa


# NLP Text cleaning

In [0]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [0]:
def text_cleaner(review):
    '''
    Clean and preprocess a review.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Lemmatize
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    review = bs.BeautifulSoup(review).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
    
    #3. Remove punctuation
    review = re.sub("[^a-zA-Z]", " ",review)
    
    #4. Tokenize into words (all lower case)
    review = review.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    review = [w for w in review if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    review = ' '.join(review+emoticons)
    # add emoticons to the end
    #7. Lemmatize
    wnl = WordNetLemmatizer()  #initialize WordNetLemmatize()
    wnl_stems = []
    token_tag = pos_tag(review.split())
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))  
        wnl_stems.append(res)
        
    review = ' '.join(wnl_stems)

    return(review)

# Get text from an array 
    csv2data(dat)
    Arguments: array

In [0]:
def csv2data(dat):    #function for cleaning text from an array
  trainx = [] 
  for i in dat:
      vcx = text_cleaner(i)
      print(vcx)
      trainx.append(vcx)
  return trainx

# Train Forest
  train_forest(X_train, y_train, xtr=None, ytr=None)
  Arguments: problems, categories, train problem, train category

In [0]:
def train_forest(X_train, y_train, xtr=None, ytr=None):

    print("Creating the bag of words model!\n")
    # CountVectorizer" is scikit-learn's bag of words tool, here we show more keywords 
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 2000) 
    
    train_bag = vectorizer.fit_transform(X_train).toarray()
    print("Training the random forest classifier!\n")
    forest = RandomForestClassifier(n_estimators = 100)       # Initialize a Random Forest classifier with 100 trees
    forest = forest.fit(train_bag, y_train)

    if xtr is None and ytr is None:
     
      train_predictions = forest.predict(train_bag)
      train_acc = metrics.accuracy_score(y_train, train_predictions)
      print("The training accuracy is: ", train_acc)

    elif xtr is not None and ytr is not None:

      test_bag = vectorizer.transform(X_test).toarray()
      train_predictions = forest.predict(train_bag)
      test_predictions = forest.predict(test_bag)
      train_acc = metrics.accuracy_score(y_train, train_predictions)
      valid_acc = metrics.accuracy_score(y_test, test_predictions)

      print("The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)

    else:
      print("Both x training and y training required")
      return None
    '''
    #X_train, X_test, y_train, y_test = train_test_split(\
    #cleaned_reviews, y, random_state=0, test_size=.2)

    # Then we use fit_transform() to fit the model / learn the vocabulary,
    # then transform the data into feature vectors.
    # The input should be a list of strings. .toarraty() converts to a numpy array
    
    train_bag = vectorizer.fit_transform(X_train).toarray()
    test_bag = vectorizer.transform(X_test).toarray()

    # You can extract the vocabulary created by CountVectorizer
    # by running print(vectorizer.get_feature_names())


    print("Training the random forest classifier!\n")
    # Initialize a Random Forest classifier with 75 trees
    forest = RandomForestClassifier(n_estimators = 50) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the target variable
    forest = forest.fit(train_bag, y_train)


    train_predictions = forest.predict(train_bag)
    test_predictions = forest.predict(test_bag)
    
    train_acc = metrics.accuracy_score(y_train, train_predictions)
    valid_acc = metrics.accuracy_score(y_test, test_predictions)
    print("The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)
    print()
    print()
    #Extract feature importnace
    print('TOP TEN IMPORTANT FEATURES:')
    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1]
    top_10 = indices[:10]
    print([vectorizer.get_feature_names()[ind] for ind in top_10]) ''' 
    y = [forest, vectorizer]
    return y

# import forest model  
   import_forest(mod, vect)
   Arguments: Model file name, vectorizer file name

In [0]:
def import_forest(mod, vect):  #Import model and vectorizer from pickle file
  f2 = open(proj_folder+vect,'rb')   
  f = open(proj_folder+mod,'rb')
  forest = pickle.load(f)   #load uing pickle
  vectorizer = pickle.load(f2)
  y = [forest, vectorizer]  
  return y

# Predict Data
  PredictData(content, fore, vecto)
  Arguments: (text, array) , forest model, forest vectorizer

In [0]:
def PredictData(content, fore, vecto):    #function for predicting data
    print("Predicting data")
    if type(content) == pd.core.series.Series or type(content) == np.ndarray or type(content)==list:  #check which tyoe of variable is this
        print("Predictiong from array")
        test_data = []
        for i in content:
            vcx = text_cleaner(i)
            test_data.append(vcx)
        test_x = vecto.transform(test_data).toarray() #transform it using vectorizer 
        prediction = fore.predict(test_x)
        return prediction
        
    elif type(content) == str or type(content) == np.str_:
        print("Predicting from string")
        test_x = vecto.transform([content]).toarray()
        prediction = fore.predict(test_x)
        return prediction
        
    else:
        print("Variable type not supported")
        print(type(content))

# Gmail API functions
    create_service() - creates a service from a token file or gets authentication using auth2. NOTE: Make sure token.pickle is 
    present in RCCB folder

In [0]:
def create_service():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(token_fl):
        with open(token_fl, 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(gmail_cred_files, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    '''   
    # Call the Gmail API for Testing
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'])'''
            
    return service

In [0]:
def create_message(sender, to, subject, message_text):

    
    """Create a message for an email.
      Args:
        sender: Email address of the sender.
        to: Email address of the receiver.
        subject: The subject of the email message.
        message_text: The text of the email message.
      Returns:
        An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    encoded_message = urlsafe_b64encode(message.as_bytes())
    return {'raw': encoded_message.decode()}

In [0]:
def send_message(service, user_id, message):
  """Send an email message.
  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.
  Returns:
    Sent Message.
  """

  message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
  print('Message Id: %s' % message['id'])
  return message


In [0]:
def send_mail(reciever, contt, categ):
  serv = create_service()
  sender = 'atulsyan850@gmail.com'  #create an email for RCCB, validate it and then change it here
  subject = categ   #Define category as the subject
  content = "Dear government,"+ os.linesep + "  This is to inform you of the complaint recieved in department " + categ + ":" + os.linesep +" "+ contt + "." + os.linesep +"            Thank you," + os.linesep + "            RCCB"
  print(content)
  rawmsg = create_message(sender, reciever, categ, content)
  send_message(serv, sender, rawmsg)

# Testing Gmail API / Sample code

In [67]:
serv = create_service()
SENDER = 'atulsyan850@gmail.com'
RECIPIENT = 'okansht@gmail.com'
SUBJECT = 'Test Message'
CONTENT = 'Hello, I do not know you! Or Do I? Sent by RCCB'

raw_msg = create_message(SENDER, RECIPIENT, SUBJECT, CONTENT)
#send_message(serv, SENDER, raw_msg)
send_mail(RECIPIENT, "I dont have water", "Water")

Dear government,
  This is to inform you of the complaint recieved in department Water:
 I dont have water.
            Thank you,
            RCCB
Message Id: 16c86fb0f4f379e3


# Testing

In [28]:
#Loading model

#Importing pretrained model
forest_files = import_forest('forest.model','forest.vectorizer')
for_model = forest_files[0]
for_vectorizer = forest_files[1]

#Importing dataset and training model
traindatafile = pd.read_csv(proj_folder+"NLP-data.tsv", header=0, \
                    delimiter="\t", quoting=3)
trn_dat_x = csv2data(traindatafile['Problem'])
fort= train_forest(trn_dat_x, traindatafile['Category'])
for2_model = fort[0]
for2_vectorizer = fort[1]

def for_run_rm(furl):     #get text and category from pretrained model
  eng_txt = GetText(furl)
  cate = PredictData(eng_txt, for_model, for_vectorizer)
  dt = [eng_txt, cate]
  return dt

def for_run_trn(flurl):   #get text and category from now trained model
  engtxt = GetText(flurl)
  ctg = PredictData(engtxt, for2_model, for2_vectorizer)
  dat_fl = [engtxt, ctg]
  return dat_fl

axc = for_run_rm(proj_folder+'NoWater.jpg')
print("Testing Pretrained model:")
print()
print("Results")
print(axc)   #Testing pretrained model

bvc = for_run_trn(proj_folder+'NoWater.jpg')
print("Testing Now Trained model:")
print()
print("Results")
print(bvc)   #Testing trained model

enough water farm
pesticide
install government tube well area else crop die
seed
large scale farmer advantage
loan farmer
less land holding hence less yield therefore low profit
get profit government scheme
government scheme reach u
bank loan landlords huge interest rate
lender also take lot money interest
request government ask moneylender provide loan low interest
give u low interest loan
give concession
please give money
cannot pay back please interest
money crisis large farmer give loan please generate employment
money even though work hour day thing pay lender also take lot money interest
give u low interest loan
please grant u loan low interest
request government ask moneylender provide loan low interest
pay work do field money lender take huge interest
please provide money low interest
request u reduce rate interest small farmer
money need agriculture interest rate kill u poor farmer
due daily wage pay paltry amount huge interest
indebted also interest rate reduce u manage day d

In [72]:
bvc[1][0]

'Agriculture'

In [73]:
send_mail(RECIPIENT,bvc[0], bvc[1][0])

Dear government,
  This is to inform you of the complaint recieved in department Agriculture:
 I do not have pain Pa.
            Thank you,
            RCCB
Message Id: 16c86fcc16466632
